In [37]:
import pandas as pd
from ast import literal_eval
import re

In [38]:
# df = pd.read_csv("../results/surechembl_smiles_canon_chiral_randomized_patents_l10p_summarizations/surechembl_summarizations_top-100_gpt-3.5-turbo_desc-3500.csv")
df = pd.read_pickle("../results/schembl_summs_v2_fixed_errors.pkl")
df["summarizations"] = df["summarizations"].apply(literal_eval)

In [39]:
print(hash(str(df['summarizations'].tolist())))

# remove trailing punctuation
df["summarizations"] = df["summarizations"].apply(lambda x: set([y.rstrip(".,;") for y in x]))
print(hash(str(df['summarizations'].tolist())))

# for each set in summarizations, remove items in set if they contain commas
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if "," not in y]))
print(hash(str(df['summarizations'].tolist())))

# convert to lowercase
df["summarizations"] = df["summarizations"].apply(lambda x: set([y.lower() for y in x]))
print(hash(str(df['summarizations'].tolist())))

# remove entries of set if they are substrings of other entries in the same set
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if not any([y in z for z in x if y != z])]))
print(hash(str(df['summarizations'].tolist())))

# if label contains format "1. text 2. text 3. text", split into individual entries. Use re
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x for y in re.split(r"\d+\.\s", y) if y != ""]))
print(hash(str(df['summarizations'].tolist())))

# if label contains format "1) text 2) text 3) text", split into individual entries. Use re
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x for y in re.split(r"\d+\)\s", y) if y != ""]))
print(hash(str(df['summarizations'].tolist())))

# remove leading text of the format "1. ", "2. ", etc.
df["summarizations"] = df["summarizations"].apply(lambda x: set([re.sub(r"^\d+\.\s", "", y) for y in x]))
print(hash(str(df['summarizations'].tolist())))

# remove leading text of the format "1) ", "2) ", etc.
df["summarizations"] = df["summarizations"].apply(lambda x: set([re.sub(r"^\d+\)\s", "", y) for y in x]))
print(hash(str(df['summarizations'].tolist())))

# split entries with escape characters on the escape characters
df['summarizations'] = df['summarizations'].apply(lambda x: set([y for y in x for y in re.split(r"[\n\t\r]", y) if y != ""]))
print(hash(str(df['summarizations'].tolist())))

# remove leading text of the form "- "
df["summarizations"] = df["summarizations"].apply(lambda x: set([re.sub(r"^- ", "", y) for y in x]))
print(hash(str(df['summarizations'].tolist())))


# remove entries in set if they contain more than 8 words
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if len(y.split()) <= 8]))
print(hash(str(df['summarizations'].tolist())))

# remove entries in a set with 2 or less characters
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if len(y) > 2]))
print(hash(str(df['summarizations'].tolist())))

# replace all escape characters with empty string
df["summarizations"] = df["summarizations"].apply(lambda x: set([re.sub(r"[\n\t\r]", "", y) for y in x]))
print(hash(str(df['summarizations'].tolist())))

# remove empty strings
df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if y != ""]))
print(hash(str(df['summarizations'].tolist())))

-8420501649113966315
2351197857673703597
2465357283314251243
1540981436122979625
-4541084522230060909
-1321147519010143855
-5085373437085930637
6321550207240105931
-6390106978444566497
6131828979895947812
-592742929922600312
2950481533017612369
-4336050897888990447
1274900590018318641
-4076461964592167797


In [40]:
# all_terms = set()
# for i in df["summarizations"]:
#     all_terms.update(i)
# # check if any entries in post_set are substrings of other entries in post_set
# # if so, remove the substring entry
# all_terms_parent_strings = set([y for y in all_terms if not any([y in z for z in all_terms if y != z])])

# # create mapping from post_set differences to post_set_ parent terms
# post_set_diffs = all_terms.difference(all_terms_parent_strings)
# post_set_diffs_map = {i: [j for j in all_terms_parent_strings if i in j][0] for i in post_set_diffs}
# # k,v pairs that differ by a trailing s
# post_set_diffs_map_plurals = {k: v for k, v in post_set_diffs_map.items() if k+"s" == v}

# # fix terms that should be plural
# df["summarizations"] = df["summarizations"].apply(lambda x: set([post_set_diffs_map[i] if i in post_set_diffs_map.keys() else i for i in x]))

In [41]:
# # remove pairs in post_set_diffs_map that are in post_set_diffs_map_plurals
# post_set_diffs_map = {k: v for k, v in post_set_diffs_map.items() if k not in post_set_diffs_map_plurals.keys()}
# # print each kv pair
# for k, v in post_set_diffs_map.items():
#     print(f"{k} -> {v}")

In [42]:
# bad_words = {
#     "therapy",
#     "steroisomer",
#     "steroisomers",
#     "diastereoisomer",
#     "diastereoisomers",
#     "modulator",
#     "modulators",
#     "pharmaceutical",
#     "pharmaceuticals",
#     "therapeutic",
#     "therapeutics",
#     "medicament",
#     "medicaments",
# }

# # remove terms if they are contained in the set of bad words
# df["summarizations"] = df["summarizations"].apply(lambda x: set([y for y in x if y not in bad_words]))

In [43]:
# create final set of terms
all_terms = set()
for i in df["summarizations"]:
    # ignore empty summarizations
    if i:    
        all_terms.update(i)

In [44]:
df[["smiles", "cid", "patent_ids", "summarization_sources", "summarizations"]].to_csv("../results/schembl_summs_v3_alg_cleaned_labels.csv", index=False)
df[["smiles", "cid", "patent_ids", "summarization_sources", "summarizations"]].to_pickle("../results/schembl_summs_v3_alg_cleaned_labels.pkl")

# save set of all terms to file
with open("../results/schembl_summarizations_vocab_v3_alg_cleaned.txt", "w") as f:
    f.write("\n".join(all_terms))

In [45]:
all_terms

{'nitroxide initiators',
 'hepatitis b virus infections',
 'aromatic character',
 'serotonin 4 receptor stimulant',
 'improved emission',
 'carbocycle derivatives',
 'biphenylyloxy',
 'glycolysis inhibitor',
 'tetragalnac ligands',
 'herg activators',
 'α4β7 antagonist',
 'tetrahydrobenzo[g]isoquinolines',
 'shp2 pathway inhibitor',
 'water resistance',
 'c-n bond formation',
 'neuromedin u receptor agonist',
 'pyrazolines',
 'small molecule ligand identification',
 'cell proliferative disorder treatment',
 'hydrazine synthesis',
 'dopamine derivative',
 'ar degradation',
 'transcription factor modulators',
 'heat generation',
 'muscle wasting',
 '4-aminoquinazolines',
 'novel synthesis method',
 'carbonless replication',
 'adamantane lactone',
 'curing range',
 'beta-adrenergic antagonist',
 'organic metal complex',
 'crop growth',
 'immunological conditions',
 'derivatized alkanolamines',
 'rorγ modulator',
 'transcription modulator',
 'genitourinary treatment',
 'cytokine production